<a href="https://colab.research.google.com/github/S-Kaito/s-kaito.github.io/blob/master/notebook/seminar/2020_0529.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import struct
import numpy as np
import codecs
from PIL import Image
import numpy as np
import math
from fractions import Fraction


def read_picture(path):
    """
    Read picture from file

    Parameter
    ---------
    path: string
        file path
    
    return
    ------
    size: (width: int, height: int)
        picture size
    max_n: int
        max value
    data: NdArray<NdArray<int>>
        picture data
    """
    
    # 画像を開く(Header情報用)
    infile = codecs.open(path, 'r', "utf-8", "ignore")
    # 画像を開く(画像データ用)
    img = np.array(Image.open(path))

    infile.readline()
    infile.readline()
    width, height = list(map(int, infile.readline().split()))
    max_n = int(infile.readline())

    infile.close()
    return (width, height), max_n, img

def write_picture(path, size, n, data):
    """
    Write picture to file
    Parameter
    ---------
    path: string
        file path
    size: (width: int, height: int)
        picture size
    n: int
        max value
    data: Array<Array<[r: int, g: int, b: int]>>
        picture data
    """

    # 画像を書き出す
    pilImg = Image.fromarray(np.uint8(data.reshape((size[0], size[1]))))
    pilImg.save(path)

def scale_up(img, value = 2):
    """
    Parameter
    ---------
    img: ndarray
    value: int
    """

    # 入力画像サイズ
    size = img.shape

    # 出力する画像の格納用配列を作成する
    # サイズ * 2 + 1で作成する
    # (+1 は端の処理を手軽に行えるように)
    ni = np.array([[0 for i in range(size[0] * value + 1)] for j in range(size[1] * value + 1)])
    # 出力画像サイズ
    shape = ni.shape

    # 出力用配列に１個飛ばしで入力画像を代入する
    for i in range(0, shape[0], value):
        for j in range(0, shape[1], value):
            x = i // value
            y = j // value

            # 右端・下橋のみ一つ左・上から取得するように
            if x == size[0]:
                x = size[0] - value
            if y == size[1]:
                y = size[1] - value
            ni[i][j] = img[x][y]

    # 縦の分の補完を行う
    for i in range(0, shape[0], value):
        for j in range(1, shape[1], value):
            ni[i][j] = (ni[i][j - 1] + ni[i][j + 1]) / value

    # 横の分の補完を行う
    for i in range(1, shape[0], value):
        for j in range(0, shape[1], value):
            ni[i][j] = (ni[i - 1][j] + ni[i + 1][j]) / value

    # 斜めの分の補完行う
    for i in range(1, shape[0], value):
        for j in range(1, shape[1], value):
            ni[i][j] = (ni[i - 1][j - 1] + ni[i + 1][j + 1]) / value
    return ni[0:-1,0:-1]

def scale_down(img, value = 2):
    """
    Parameter
    ---------
    img: ndarray
    value: int
    """

    # 入力画像サイズ
    size = img.shape
    # 出力する画像の格納用配列を作成する
    # サイズ // 2 で作成する
    ni = np.array([[0 for i in range(size[0] // value)] for j in range(size[1] // value)])
    # 出力画像サイズ
    shape = ni.shape

    # 平均操作法で画素を代入していく
    for i in range(0, shape[0]):
        for j in range(0, shape[1]):
            ni[i][j] = np.average(img[i * value: i * value + value, j * value: j * value + value])

    return ni

def rotate(img, value = 30):
    # ラジアンに変換する
    rad = math.radians(-1 * value)

    # 回転後の位置の計算用の行列を宣言
    cof = np.array([
        [math.cos(rad), math.sin(rad)],
        [-1 * math.sin(rad), math.cos(rad)]
    ])

    # 画像の上下左右を求める
    max_x, max_y = 0, 0
    max_x = math.ceil(max([np.dot(cof, np.array([[0], [0]]))[0], np.dot(cof, np.array([[0], [img.shape[1]]]))[0], np.dot(cof, np.array([[img.shape[0]], [0]]))[0], np.dot(cof, np.array([[img.shape[0]], [img.shape[1]]]))[0]]))
    max_y = math.ceil(max([np.dot(cof, np.array([[0], [0]]))[1], np.dot(cof, np.array([[0], [img.shape[1]]]))[1], np.dot(cof, np.array([[img.shape[0]], [0]]))[1], np.dot(cof, np.array([[img.shape[0]], [img.shape[1]]]))[1]]))
    min_x = math.floor(min([np.dot(cof, np.array([[0], [0]]))[0], np.dot(cof, np.array([[0], [img.shape[1]]]))[0], np.dot(cof, np.array([[img.shape[0]], [0]]))[0], np.dot(cof, np.array([[img.shape[0]], [img.shape[1]]]))[0]]))
    min_y = math.floor(min([np.dot(cof, np.array([[0], [0]]))[1], np.dot(cof, np.array([[0], [img.shape[1]]]))[1], np.dot(cof, np.array([[img.shape[0]], [0]]))[1], np.dot(cof, np.array([[img.shape[0]], [img.shape[1]]]))[1]]))
    # 無駄に大きい配列を宣言
    ni = np.array([[0 for i in range(5000)] for j in range(5000)])
    
    # 配列の真ん中に回転後の画素を代入していく
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            a = np.dot(cof, np.array([[i], [j]]))
            ni[math.floor(a[0]) + 2500, math.floor(a[1]) + 2500] = img[i][j]

    # 無駄に大きい配列から画像の部分だけ取得
    return ni[min_x + 2500: max_x + 2500, min_y + 2500: max_y + 2500]

In [0]:
PATH = "star.pgm"

# 画像を読み込む
size, n, pic = read_picture(PATH);

# 画像の縮小処理を行う
pic_s = scale_down(pic, 2)
write_picture("output_s.pgm", (size[0] // 2, size[1] // 2), n, pic_s)

# 画像の拡大処理を行う
pic_b = scale_up(pic, 2)
write_picture("output_b.pgm", (size[0] * 2, size[1] * 2), n, pic_b)

# 画像の回転処理を行う
pic_r = rotate(pic, 30)
write_picture("output_r.pgm", (pic_r.shape[0], pic_r.shape[1]), n, pic_r)

# 無事終了したことを知らせる
print("Finish!")

Finish!
